In [1]:
import sys
import os
import numpy as np
import time
from imp import reload

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
    
from testing.inputs import *

new directory is: C:\Users\Luke\Desktop\Barracuda\AutomatedCE


## Step 1. Load the CE System with a Config File

Select one of the predesigned configurations (TE300, Eclipse Ti, Simulation). See an example config under "/config/test-system.cfg". 

Change the variable CONFIG_PATH to the absolute path where the config file resides. 

Be sure to make the string a raw string by prepending *'r'*. **Example:** r"C:\path\to\config\"*

In [3]:
CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()
ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L1\MicroControlServer.py


 Change System Config setting if thorlabs is needed. 


{'outlet_pressure': <L2.PressureControl.ArduinoPressure object at 0x0000024B5A4FCB48>, 'xy_stage': <L2.XYControl.PycromanagerXY object at 0x0000024B5A4FCA88>, 'objective': <L2.ZControl.PycromanagerZ object at 0x0000024B5A4FCC08>, 'outlet_z': <L2.ZControl.ArduinoZ object at 0x0000024B5A4FCC88>, 'inlet_z': <L2.ZControl.ArduinoZ object at 0x0000024B5A4FCB88>, 'filter_wheel': None, 'excitation_shutter': None, 'camera': <L2.CameraControl.PycromanagerControl object at 0x0000024B5A4FCE88>, 'high_voltage': <L2.HighVoltageControl.SpellmanPowerSupply object at 0x0000024B5A4FCD88>, 'detector': <L2.DetectorControl.PhotomultiplierDetector object at 0x0000024B5A500208>}


# Step 2. Load the Template File

The template file tells the program where to move the XY stage for each well as well as how high to raise the capillary inlet during a move. Change the variable TEMPLATE_PATH to the absolute path where the template.txt file resides. 

In [4]:
TEMPLATE_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\80WellTest.txt"
from L4 import AutomatedControl
reload(AutomatedControl)

auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)

# Step 3. Calibrate the Capillary Inlet

The method for calibration depends on the z-stage used to raise the capillary inlet. Select the correct route for the hardware employed. 

## Thorlabs Kinesis Z Stage

### Step 3.1 Lower the Capillary (thorlabs)
Move the capillary to a safe location where it will not be crushed if the stage lowers too far. Using the controls on the Z-stage itself, lower the capillary till as far as it will go. 


In [8]:
resp = get_yes_no("Have you lowered the Kinesis Z stage to its lowest position? (y/n):  ")
if not resp:
    print("Please make sure the Kinesis stage is lowered to its lowest position before continuing")
else:
    print("Proceed to the next cell.")
    

Have you lowered the Kinesis Z stage to its lowest position? (y/n):  y
Proceed to the next cell.


### Step 3.2 Home the Capillary (thorlabs)

Thorlabs needs to register the home position. In the next cell we will call the go_home command to home the capillary. 

In [9]:
ce_system.inlet_z.homing()

### Step 3.1 (Arduino/Optics-Focus)
The optics focus moves the capillary all the way up during the home step. Once this is done, move the capillary to a safe position and lower (with 1-5mm increments) until the capillary is at 0 mm. 

In [8]:
ce_system.inlet_z.set_rel_z(-5)

In [12]:
resp = get_yes_no("Have you lowered the Arduino Z stage to 0 mm? (y/n):  ")
if not resp:
    print("Please make sure the Aduino stage is lowered to 1 mm position before continuing")
else:
    print("Proceed to the step 3.3 ")

Have you lowered the Arduino Z stage to 0 mm? (y/n):   y
Proceed to the step 3.3 


### Step 3.3 Adjust Capillary Position (all stages)

Adjust the capillary position by sliding the detection apparatus up or down the optical support post. The capillary should be level with the bottom of the template for single-cell assays, or the top-of the template for well-based assays. 

*Aligning to the top of the template is a minor safety feature to prevent the user from sending the capillary to Zero position unexpetedly and crashing the capillary against the bottom of the template. However, capillary needs to be able to reach at least the glass slide of the cell reservoir for single-cell assays*


In [10]:
resp = get_yes_no("Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):  ")
if not resp:
    print("Please make sure the capillary is moved to the correct height with respect to the template before continuing")
else:
    print("Proceed to the next cell")

Have you adjusted the capillary height manually so that it is flush (level) with the top or bottom of the template? (y/n):   y
Proceed to the next cell


### Step 3.4 Raise the capillary to a safe height

The next cell will raise the capillary height to a location where it ***should*** be safe for any XY movements. Verify the capillary will not crash at this new height. 

In [11]:
target_z = get_float("Please enter a safe height to move the stage to: ")
# Move the capillary up and wait for it to stop
ce_system.inlet_z.set_z(target_z)
ce_system.inlet_z.wait_for_target(target_z)

resp = get_yes_no("Is the capillary at a safe location?: ")
if resp:
    print("Continue to next step")
else:
    print("Please raise the capillary to an appropriate height")

Please enter a safe height to move the stage to:  23
Is the capillary at a safe location?:  y
Continue to next cell


# Step 4. Home the XY Stage

This next step provides the calibration for the XY axis for the XY stage and capillary inlet. 


### Step 4.1 (Nikon Eclipse)

The nikon eclipse has a home position defined to the center of the template. It is best to use this as the home position. For the capillary alignment, align the capillary concentrically with the objective. The capillary should be within 0.5 mm of the center of the objective if using PCR tubes at any point. 

In [12]:
resp = get_yes_no("Is the capillary aligned with the objective?")
if resp:
    print("Continue to next step")
else:
    print("Please raise align the capillary with the objective")

Is the capillary aligned with the objective? y
Continue to next step


# Step 5 Verify your calibration

Steps 3-4 calibrate the capillaries position with the microscope's position. Now we need to verify that our alignment is correct. To do this we will move the capillary to a well and make sure that it is centered over the well. Then we will lower the capillary to a height specified by the method to make sure the capillary can reach the liquid. 


### Step 5.2 Raise the capillary to a safe height 

In [5]:
z = get_float("Set the capillary to a safe travel height: ")
ce_system.inlet_z.set_z(z)
print("Proceed to next cell")

Set the capillary to a safe travel height:  23
Proceed to next cell


### Step 5.2 Center the Capillary over a well

In [13]:
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
ce_system.xy_stage.set_xy(well.xy)
resp = input("Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.")


Please enter the well name to move to (press enter to see a list): k
Please select one of the following wells:
calibrate
w_a0
w_a1
w_a2
w_a3
w_a4
w_a5
w_a6
w_a7
w_b0
w_b1
w_b2
w_b3
w_b4
w_b5
w_b6
w_b7
w_c0
w_c1
w_c2
w_c3
w_c4
w_c5
w_c6
w_c7
w_d0
w_d1
w_d2
w_d3
w_d4
w_d5
w_d6
w_d7
w_e0
w_e1
w_e2
w_e3
w_e4
w_e5
w_e6
w_e7
w_f0
w_f1
w_f2
w_f3
w_f4
w_f5
w_f6
w_f7
w_g0
w_g1
w_g2
w_g3
w_g4
w_g5
w_g6
w_g7
w_h0
w_h1
w_h2
w_h3
w_h4
w_h5
w_h6
w_h7
w_i0
w_i1
w_i2
w_i3
w_i4
w_i5
w_i6
w_i7
w_j0
w_j1
w_j2
w_j3
w_j4
w_j5
w_j6
w_j7
Please enter the well name to move to (press enter to see a list): w_j6
Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.


### Step 5.3 Lower the capillary to the height described in the method file


In [14]:
height_z = get_float("What height should the capillary be lowered to?")
ce_system.inlet_z.set_z(height_z)
resp = get_yes_no("Is the capillary touching the liquid in this well? ")
if not resp:
    print("Adjust the height of the capillary manually (and carefully!) so that the capillary is touching the liquid")
else:
    print("Calibration Finished")

What height should the capillary be lowered to? 6.5
Is the capillary touching the liquid in this well?  y
Calibration Finished
